In [6]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import collections
import re
import pprint as pp
import numpy as np
import matplotlib.pyplot as plt

import math
import gzip
import pickle as pkl
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import random
random.seed(123)
import os
import time
import tflearn

import multiprocessing as mp
from multiprocessing.pool import ThreadPool
from multiprocessing import Process

from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization, LeakyReLU
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras import losses

from sklearn.metrics import precision_recall_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_recall_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression as LR
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer 

import tensorflow as tf
#tf.compat.v1.enable_eager_execution()

import itertools
import gzip
import pickle as pkl

import fonctions
import batch_processing
import trend_loading
from imblearn.under_sampling import RandomUnderSampler 
import metric_loading


from os import listdir
from tqdm.notebook import tqdm
from os.path import isfile, join

from scipy.sparse import hstack
from keras import backend as K
from sklearn.metrics import log_loss
from tensorflow import ones_like, equal, log
from keras import backend as K
from tensorflow.python import mul
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def read_batch(directory, doc_name):
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep=',', header=0, index_col=False,low_memory=False)
    df = df.drop(df.columns[ [0] ], axis=1)
    df.index = df.tweet_id+'-'+df.engaging_user_id

    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    y=df.filter( labels )

    X = df.drop(labels + ['tweet_utctimestamp','language','tweet_id','tweet_timestamp',
                          'engaged_with_user_account_creation', 'text_tokens',
                          'present_links','present_domains','hashtags',
                          'engaging_user_account_creation', 'present_media',
                          'engaged_with_user_id', 'engaging_user_id' ],axis=1)
        
    #nan_cols = [i for i in X.columns if X[i].isnull().any() ]
    #test = [ X[var].fillna( 0, inplace=True) for var in nan_cols ]
    
    X = X.drop(['mean_daily_user_presence','std_daily_user_presence','author_engagement', 'author_ratio_like', 'author_ratio_retweet',
                 'author_ratio_rtc', 'author_ratio_reply', 'user_engagement', 
                 'user_ratio_like', 'user_ratio_retweet', 'user_ratio_rtc', 'user_ratio_reply'],axis=1)
    
    return X, y

def build_NN(nb):
    
    classifier = Sequential()

    classifier.add(Dense( 200, input_dim=nb ,activation='relu' ))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.3))
    classifier.add(Dense( 100 ,activation='relu' ))
    classifier.add(Dense( 100 ,activation='relu' ))
    classifier.add(Dense( 50 ,activation='relu' ))
    classifier.add(Dense( 20 ,activation='relu' ))
    classifier.add(Dense(1, activation='sigmoid'))
    
    opt = optimizers.Adam()
    classifier.compile(optimizer = opt, loss='binary_crossentropy') 

    return classifier


def save_model(target, model_id, classifier):
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/NN_{}_{}.pkl.gz'.format(target,model_id),'wb') as f:
        pkl.dump(classifier,f)
    return True

def train_on_chunk(target, chunk, model_id, classifier, nb_epochs):

    sc = StandardScaler()
    
#     weights = {'like_timestamp':{0:1,1:1}, 'retweet_timestamp':{0:1,1:1},
#            'retweet_with_comment_timestamp':{0:1,1:1},'reply_timestamp':{0:1, 1:1}  }
    weights = {'like_timestamp':{0:0.88,1:1.16}, 'retweet_timestamp':{0:0.55,1:5},
           'retweet_with_comment_timestamp':{0:0.50,1:50},'reply_timestamp':{0:0.51, 1:25}  }

    directory = '/home/maxime/Desktop/RecSys2020/data/processed_batches/'

    for _ in range(nb_epochs):

        iteration=1
        for batch_file in chunk:

            print(iteration)

            X, y = read_batch(directory, batch_file)
            y=y[target]

            
            X_train, y_train = shuffle(X, y, random_state=0)
            X_train = sc.fit_transform(X_train) 
            
            classifier.fit(X_train, y_train, verbose=0,class_weight=weights[target])
        
            iteration=iteration+1
            
    save_model(target, model_id, classifier)

    return True

In [11]:
batch_path='/home/maxime/Desktop/RecSys2020/data/processed_batches/'
chunk = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]

targets = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']

classifier = build_NN(91)
model_id = 58

if __name__ == '__main__':
    
    #Setup a list of processes that we want to run 
    processes = [ mp.Process(target=train_on_chunk, args=(target, chunk, model_id, classifier, 1) ) for target in targets ]

    #Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join()

1
1
1
1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
2
2
2
2
3
3
3
3
4
4
4
4
5
5
5
5
6
6
6
6
7
7
7
7
8
8
8
8
9
9
9
9
10
10
10
10
11
11
11
11
12
12
12
12
13
13
13
13
14
14
14
14
15
15
15
15
16
16
16
16
17
17
17
17
18
18
18
18
19
19
19
19
20
20
20
20
21
21
21
21
22
22
22
22
23
23
23
23
24
24
24
24
25
25
25
25
26
26
26
26
27
27
27
27
28
28
28
28
29
29
29
30
29
30
30
31
30
31
31
32
31
32
32
33
33
32
33
34
34
33
34
35
35
34
35
36
36
35
36
37
37
36
37
38
38
37
38
39
39
38
39
40
40
39
40
41
41
40
41
42
42
41
43
42
43
42
44
43
44
43
45
44
45
44
46
45
46
45
47
47
46
46
48
48
47
47
49
49
48
48
50
50
49
49
51
51
50
50
52
52
51
51
53
53
52
52
54
54
53
53
55
55
54
54
56
56
55
55
57
57
56
56
58
58
57
57
59
59
58
58
60
60
59
59
61
61
60
60
62
62
61
61
63
63
62
62
64
64
63
63
65
65
64
64
66
66
65
65
67
67
66
66
68
68
67
67
69
69
68
68
70
70
69
69
71
71
70
7

479
473
464
480
480
474
481
481
465
475
482
482
466
476
483
483
467
477
484
484
468
478
485
485
469
479
486
486
470
480
487
487
481
471
488
488
482
472
489
489
483
473
490
490
484
474
491
491
485
475
492
492
486
476
493
493
487
477
494
494
488
478
495
495
489
479
496
496
490
480
497
497
491
481
498
498
492
482
499
499
493
483
500
500
494
484
495
485
496
486
497
487
498
488
499
489
500
490
491
492
493
494
495
496
497
498
499
500


In [4]:
##### CALIBRATION OF THE CLASSIFIER

def classifier_calibration(directory, target, model_id, new_chunk, sc):
    
    print('init')
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/NN_{}_{}.pkl.gz'.format(target,model_id), 'rb') as f:
        classifier = pkl.load(f)
        loss_history = pkl.load(f)
        metrics_history = pkl.load(f)
        
    print('model loaded')
    
    X_agreg, y_agreg =fonctions.agregated_dataset(directory,target, new_chunk, balanced=False)
    X_agreg = sc.fit_transform(X_agreg)
    
    print('data agregated')

    y_pred = classifier.predict_proba(X_agreg)

    lr = LR()                                                       
    lr.fit( y_pred[:,0].reshape(-1, 1), y_agreg )

    iteration=1

    for batch_file in new_chunk:
        print(iteration)
        X, y = batch_processing.read_batch(directory, batch_file, False)
        X = sc.fit_transform(X)
        X_train, X_val, y_train, y_val = train_test_split(X, y[target], test_size=0.2, random_state=0)
    
        y_init = classifier.predict_proba(X_val)
        y_hat = lr.predict_proba( y_init[:,0].reshape( -1, 1 ))[:,1]
    
        y_hat = y_hat.astype('float64')
        y_init = y_init.astype('float64')
        
        metrics_history.append( [fonctions.compute_prauc(y_hat,y_val), fonctions.compute_rce(y_hat,y_val)] )
        iteration=iteration+1
        
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/calibrated_NN_{}_{}.pkl.gz'.format(target,model_id),'wb') as f:
        pkl.dump(lr,f)
        pkl.dump(metrics_history,f)
  
    return True

In [5]:
targets = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
directory = '/home/maxime/Desktop/RecSys2020/data/processed_batches/'
model_id=9
sc = StandardScaler()

if __name__ == '__main__':
    # Setup a list of processes that we want to run
        
    processes = [ mp.Process(target=classifier_calibration, args=(directory, target, model_id, chunks[4][:100], sc) ) for target in targets ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 




init
init
init
init
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
model loaded
model loaded


model loaded


model loaded



0.0    0.973381
1.0    0.026619
Name: 0, dtype: float64
2400000

0.0    0.992432
1.0    0.007568
Name: 0, dtype: float64
2400000

0.0    0.888491
1.0    0.111509
Name: 0, dtype: float64
2400000

0.0    0.563235
1.0    0.436765
Name: 0, dtype: float64
2400000
data agregated
data agregated
data agregated
data agregated
1
1
2
1
2
3
2
4
3
3
5
4
4
6
5
1
5
7
6
2
6
8
7
3
7
9
8
4
8
10
9
5
9
11
10
6
10
12
11
7
11
13
12
8
12
14
13
9
13
15
14
10
16
14
15
11
17
15
16
12
18
17
16
13
19
18
17
14
20
19
18
15
21
20
19
22
16
21
20
23
17
22
21
18
24
23
22
19
25
24
23
20
26
25
27
24
21
26
22
28
25
27
23
29
26
28
24
30
29
27
25
31
30
28
26
32
31
29
27
33
32
30
28
34
33
31
29
35
34
32
30
36
35
33
37
31
36
34
32
38
37
35
33
39
38
36
34
40
39
37
35
41
40
38
36
42
41
37
39
43
42
38
40
44
43
39
41
45
44
40
46
42
45
41
47
43
46
42
48
44
47
43
49
45
48
44
50
49
46
45
51
50
47
46
52
51
48
47
53
52
49
48
54
53
50
49
54
55
51
50
55
56
52
51
56
57
53
52
57
58
53
54
58
59
54
55
59
60
55
56
60
56
61
57
61
62
57
58
63